## 上课时间分布

### 导入需要的包

In [1]:
import pymysql
import pandas as pd
from datetime import datetime

### 连接数据库

In [2]:
def data_out_sql(sql):
    #conn = pymysql.connect(host=DB['host'], port=DB['port'], user=DB['user'], passwd=DB['password'], db=DB['dbname'],charset='utf8')
#     conn = pymysql.connect(user='lingoace', password='DB4lingo.com', database='data_analysis')
    # 使用异步连接mysql
    conn = pymysql.connect(user='lingoace', password='DB4lingo.com', database='lingoacedb_snapshot', host='192.168.3.237')

    # 创建游标
    cursor = conn.cursor()
    # 执行sql语句
    try:
        cursor.execute(sql)
        # 调出数据
        data = cursor.fetchall()
    # cols为字段信息 例如(('factory_id', 253, None, 6, 6, 0, False), ('szDeviceId', 253, None, 30, 30, 0, False),('update_time', 7, None, 19, 19, 0, False))
    except Exception as e:
        print('select sql error, err={}'.format(e))
    cols = cursor.description
    # 执行
    conn.commit()
    conn.close()
    # 将数据truple转换为DataFrame
    col = []
    for i in cols:
        col.append(i[0])
    data = list(map(list, data))
    data = pd.DataFrame(data,columns=col)

    #print(data)
    return data

### 数据转储cvs配置

In [3]:
#常用数据存放目录
catelog='./data/'
#临时数据存放目录
tempData='./temp/'
#数据库数据数据定义
Database='lingoacedb'

def data_to_csv(df, name):
    
    catelog='./data/'

    name_suffix = datetime.now().strftime('%Y%m%d')
    
    df.to_csv('%s%s%s.csv' % (catelog, name, name_suffix), encoding='utf_8_sig')
    

In [ ]:
user_enter_class_sql = '''select 
    MONTH(CONVERT_TZ(svm.enter_time, '+00:00', '+08:00')) as month, 
    DATE_FORMAT(CONVERT_TZ(svm.enter_time, '+00:00', '+08:00'), '%W') as week, 
    HOUR(CONVERT_TZ(svm.enter_time, '+00:00', '+08:00')) as hour, 
    count(*) as access_count
from schedule_virtualclass_member svm 
left join classroom_virtualclass_info cvi on cvi.id=svm.virtual_class_id
where cvi.class_type_id<>3 and 
svm.enter_time is not null and YEAR(CONVERT_TZ(svm.enter_time, '+00:00', '+08:00'))=2020
group by DATE_FORMAT(CONVERT_TZ(svm.enter_time, '+00:00', '+08:00'), '%W'), 
         HOUR(CONVERT_TZ(svm.enter_time, '+00:00', '+08:00')), 
         MONTH(CONVERT_TZ(svm.enter_time, '+00:00', '+08:00'))
order by  MONTH(CONVERT_TZ(svm.enter_time, '+00:00', '+08:00')) desc, 
          DATE_FORMAT(CONVERT_TZ(svm.enter_time, '+00:00', '+08:00'), '%W') desc, 
          HOUR(CONVERT_TZ(svm.enter_time, '+00:00', '+08:00')) desc'''
access_data= data_out_sql(user_enter_class_sql)
# access_data.describe()
data_to_csv(access_data,'useraccess')
access_data.head()

### 数据可视化

In [ ]:
import altair as alt

In [ ]:
base0=alt.Chart(access_data).mark_line(point=True,color="red").encode(
    x='hour',
    y='access_count',
    color= 'week:N',
    column='month'
).properties(
    width=800
)
base0

### python pandas处理数据

In [ ]:
student_access_time_sql = '''select svm.enter_time as enter_time from schedule_virtualclass_member svm 
where svm.start_time>'2019-12-31 16:00:00' and svm.enter_time is not null'''

student_access_time_data = data_out_sql(student_access_time_sql)

In [ ]:
tutor_salary_sql = '''select date_format(CONVERT_TZ(cvi.start_time,'+00:00','+08:00'), '%Y%m') as date, uti.username, uti.real_name,uti.identity_name, cvi.tutor_user_id, sum(virtualclass_amount.class_amount), count(virtualclass_amount.class_amount), virtualclass_amount.class_amount from
(select sum(amount) as class_amount, reference from finance_balance_change where reason in (7,8, 113, 114)
group by reference) virtualclass_amount
left join classroom_virtualclass_info cvi on cvi.id=virtualclass_amount.reference
left join user_tutor_info uti on uti.id=cvi.tutor_user_id
where date_format(CONVERT_TZ(cvi.start_time,'+00:00','+08:00'), '%Y%m') in ('202001', '202002', '202003')
group by date_format(CONVERT_TZ(cvi.start_time,'+00:00','+08:00'), '%Y%m'), cvi.tutor_user_id, virtualclass_amount.class_amount
order by date_format(CONVERT_TZ(cvi.start_time,'+00:00','+08:00'), '%Y%m')  desc'''
tutor_salary = data_out_sql(tutor_salary_sql)

In [ ]:
data_to_csv(tutor_salary,'tutor_salary_detail')